In [ ]:
import pandas as pd
import numpy as np

# Preprocess

In [ ]:
Y_preds = pd.read_csv('pred_y.csv')
Y_preds['alg'] = Y_preds['alg'].str.replace('deep ', 'deep$')
Y_preds['alg'] = Y_preds['alg'].str.replace('Neural Net SVM ', 'Neural Net SVM$')
Y_preds[['clf','alg_matcher']] = Y_preds['alg'].str.split('$', expand=True)
Y_preds = Y_preds.drop(['alg', 'Unnamed: 0'], axis=1)

In [ ]:
eval_preds = pd.read_csv('pred_eval.csv')
eval_preds['P_hat'] = eval_preds['P_hat'].str.replace('[', '').str.replace(']', '').astype(float)
eval_preds['F_hat'] = eval_preds['F_hat'].str.replace('[', '').str.replace(']', '').astype(float)
eval_preds['alg'] = eval_preds['alg'].str.replace('deep ', 'deep$')
eval_preds['alg'] = eval_preds['alg'].str.replace('SVM ', 'SVM$')
eval_preds[['reg','alg_matcher']] = eval_preds['alg'].str.split('$', expand=True)
eval_preds = eval_preds.drop(['alg', 'Unnamed: 0'], axis=1)

In [ ]:
df = pd.merge(left=Y_preds, right=eval_preds)
df['alg'] = [' '.join(i) for i in zip(df['alg_matcher'].map(str), df['clf'], df['reg'])]
alg_matches = pd.read_csv('../../algs.csv')
labels = pd.read_csv('../../Excel2CIDX.csv', header = None, names = ['candName', 'targName', 'real'])
labels['correspondence'] = list(zip(labels.targName.str.replace(' ', ''), labels.candName.str.replace(' ', '')))
labels['correspondence'] = labels['correspondence'].astype(str)
alg_matches['correspondence'] = list(zip(alg_matches.targName, alg_matches.candName))
alg_matches['correspondence'] = alg_matches['correspondence'].astype(str)
alg_matches = alg_matches.merge(labels[['correspondence', 'real']], how='left', on = 'correspondence').fillna(0.0)

In [ ]:
def evaluate(df, name):
    matchers = df['matcher'].unique().tolist()
    algs = df['alg'].unique().tolist()
    all_correct = 65
    res = pd.DataFrame(columns=['Algorithmic Matcher', 'Classifier', 'Regressor', 'matcher', 'P', 'R', 'F'])
    row_i = 1
    for alg in algs:
        if not alg:
            continue
        for matcher in matchers:
            sum_correct = len(df[(df['alg'] == alg)
                                 & (df['matcher'] == matcher)
                                 & (df['pred'] == df['real'])
                                 & (df['real'] == 1)])
            sum_answered = len(df[(df['alg'] == alg)
                                 & (df['matcher'] == matcher)
                                 & (df['pred'] == 1)])
            P = sum_correct/sum_answered if sum_answered > 0 else np.nan
            R = sum_correct/all_correct
            F = (2*P*R)/(P+R) if (P+R) > 0 else np.nan
            alg_matcher = df.loc[(df['alg'] == alg) & (df['matcher'] == matcher), 'alg_matcher'].tolist()[0]
            clf = df.loc[(df['alg'] == alg) & (df['matcher'] == matcher), 'clf'].tolist()[0]
            reg = df.loc[(df['alg'] == alg) & (df['matcher'] == matcher), 'reg'].tolist()[0]
            res.loc[row_i] = np.array([alg_matcher,
                                       clf,
                                       reg,
                                       matcher,
                                       P,
                                       R,
                                       F])
            row_i += 1
        res[['P','R','F']]=res[['P','R','F']].astype(float)
        pd.DataFrame(res.groupby(['1LM + 2LM', 'Classifier', 'Regressor'])[['P','R','F']].mean()).to_csv('res/' +name+ '.csv')
    return res

# Unbaised Matching

In [ ]:
# df_conf = df.loc[df['alg'] == 'Ontobuilder Term Match Neural Net SVM SVM', ['matcher', 'correspondence', 'conf', 'real']]
# df_conf['index'] = df_conf.index

In [ ]:
# df_conf = df_conf.assign(decision_count = df_conf.groupby('matcher')['conf'].transform('count'))
# df_conf['conf_sum'] = df_conf['conf']
# sum_confs = df_conf.groupby(['matcher', 'index'])['conf_sum'].sum().groupby(level=0).cumsum().reset_index().set_index(['index'])
# df_conf = df_conf.drop(['conf_sum', 'matcher'], axis = 1)
# df_conf = pd.merge(left=df_conf, right=sum_confs, on = ['index'])

In [ ]:
# df_conf['pred_recall'] = 1
# df_conf['pred_P_static'] = 0
# df_conf.loc[df_conf['conf'] >= 1.0, 'pred_P_static'] = 1
# df_conf['pred_F_static'] = 0
# df_conf.loc[df_conf['conf'] >= 0.5, 'pred_F_static'] = 1

In [ ]:
# matchers = df_conf['matcher'].unique().tolist()
# df_conf['pred_P_dynamic'] = 0
# df_conf['pred_F_dynamic'] = 0
# for m in matchers:
#     H = df_conf[df_conf['matcher'] == m]
#     sum_conf_P = 0
#     match_size_P = 0
#     sum_conf_F = 0
#     match_size_F = 0
#     all_correct = 65
#     for _, h in H.iterrows():
#         curr_P = sum_conf_P/match_size_P if match_size_P > 0 else 0.0
#         curr_F = (2*sum_conf_F)/(match_size_F + all_correct)
#         if h['conf'] >= curr_P:
#             df_conf.loc[df_conf['index'] == h['index'], 'pred_P_dynamic'] = 1
#             sum_conf_P += h['conf'] 
#             match_size_P += 1
#         if h['conf'] >= 0.5*curr_F:
#             df_conf.loc[df_conf['index'] == h['index'], 'pred_F_dynamic'] = 1
#             sum_conf_F += h['conf'] 
#             match_size_F += 1

In [ ]:
# df_conf['alg'] = 'dummy'

In [ ]:
# unbiased_target_R = evaluate(df_conf.rename(columns={'pred_recall': 'pred'}), 'unbiased_target_R')
# unbiased_target_P_static = evaluate(df_conf.rename(columns={'pred_P_static': 'pred'}), 'unbiased_target_P_static')
# unbiased_target_F_static = evaluate(df_conf.rename(columns={'pred_F_static': 'pred'}), 'unbiased_target_F_static')
# unbiased_target_P_dynamic = evaluate(df_conf.rename(columns={'pred_P_dynamic': 'pred'}), 'unbiased_target_P_dynamic')
# unbiased_target_F_dynamic = evaluate(df_conf.rename(columns={'pred_F_dynamic': 'pred'}), 'unbiased_target_F_dynamic')

# History Processing

In [ ]:
df_hp = df.copy()
df_hp = df_hp[df_hp['matcher'] == 2]

In [ ]:
df_hp['pred_R'] = 1
df_hp['pred_P_static'] = 0
df_hp.loc[df_hp['pred_conf'] >= 0.95, 'pred_P_static'] = 1
df_hp['pred_F_static'] = 0
df_hp.loc[df_hp['pred_conf'] >= 0.5, 'pred_F_static'] = 1

In [ ]:
hp_vanila = evaluate(df_hp, 'vanila')
hp_target_R = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_R': 'pred'}), 'hp_pred_R')
hp_target_P_static = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_P_static': 'pred'}), 'hp_target_P_static')
hp_target_F_static = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_F_static': 'pred'}), 'hp_target_F_static')

In [ ]:
df_hp['pred_P_dynamic'] = 0
df_hp.loc[df_hp['pred_conf'] >= df_hp['P_hat'], 'pred_P_dynamic'] = 1
df_hp['pred_F_dynamic'] = 0
df_hp.loc[df_hp['pred_conf'] >= 0.5*df_hp['F_hat'], 'pred_F_dynamic'] = 1

In [ ]:
hp_target_P_dynamic = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_P_dynamic': 'pred'}), 'hp_target_P_dynamic')
hp_target_F_dynamic = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_F_dynamic': 'pred'}), 'hp_target_F_dynamic')

In [ ]:
df_hp['pred_P_real'] = 0
df_hp.loc[df_hp['pred_conf'] >= df_hp['P_hat'], 'pred_P_real'] = 1
df_hp['pred_F_real'] = 0
df_hp.loc[df_hp['pred_conf'] >= 0.5*df_hp['F_hat'], 'pred_F_real'] = 1

In [ ]:
hp_target_P_real = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_P_real': 'pred'}), 'hp_target_P_real')
hp_target_F_real = evaluate(df_hp.rename(columns={'pred': 'temp', 'pred_F_real': 'pred'}), 'hp_target_F_real')

# Recall Boosting

In [ ]:
matchers = df_hp['matcher'].unique().tolist()
algs = df_hp['alg_matcher'].unique().tolist()
df_rb_th = {}
df_rb_maxd = {}
df_rb_maxdcol = {}
df_rb_dom = {}
# R is 0.0, F is 0.5, and P is 1.0
thresholds = np.linspace(0.0, 1.0, num=11)
thresholds[0] = 0.001
thresholds = [0, ] + [round(t, 1) for t in thresholds] + [1.001, ]
for num in thresholds:
    df_rb_th[str(num)] = pd.DataFrame(columns=df_hp.columns)
    df_rb_maxd[str(num)] = pd.DataFrame(columns=df_hp.columns)
    df_rb_maxdcol[str(num)] = pd.DataFrame(columns=df_hp.columns)
    df_rb_dom[str(num)] = pd.DataFrame(columns=df_hp.columns)
df_rb_th['P_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_th['F_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxd['P_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxd['F_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxdcol['P_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxdcol['F_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_dom['P_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_dom['F_dynamic'] = pd.DataFrame(columns=df_hp.columns)
df_rb_th['P_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_th['F_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxd['P_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxd['F_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxdcol['P_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_maxdcol['F_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_dom['P_real'] = pd.DataFrame(columns=df_hp.columns)
df_rb_dom['F_real'] = pd.DataFrame(columns=df_hp.columns)

In [ ]:
other_cols = [col for col in df_hp.columns.tolist() if col not in ['correspondence', 'real']]

In [ ]:
for alg in algs:
    if alg == 'all':
        alg = 'Ontobuilder Term Match'
    print(alg)
    for matcher in matchers:
#         SETUP:
        answered = df_hp[(df_hp['alg_matcher'] == alg) & (df_hp['matcher'] == matcher)]
        unanswered = alg_matches[~alg_matches['correspondence'].isin(answered['correspondence'].tolist())][['correspondence', alg, 'real']]
        last_P_pred = answered['P_hat'].iloc[[-1]].tolist()[0]
        last_P_real = answered['P'].iloc[[-1]].tolist()[0]
        last_F_pred = answered['F_hat'].iloc[[-1]].tolist()[0]
        last_F_real = answered['F'].iloc[[-1]].tolist()[0]
        answered[['targName', 'candName']] = answered['correspondence'].str.split(',', expand = True)
        answered['candName'] = answered['candName'].str.replace('\'', '').str.replace(')', '').str.strip()
        answered['targName'] = answered['targName'].str.replace('\'', '').str.replace('(', '').str.strip()
        unanswered[['targName', 'candName']] = unanswered['correspondence'].str.split(',', expand = True)
        unanswered['candName'] = unanswered['candName'].str.replace('\'', '').str.replace(')', '').str.strip()
        unanswered['targName'] = unanswered['targName'].str.replace('\'', '').str.replace('(', '').str.strip()
        max_rows = answered.groupby('candName')['pred'].max().to_frame(name = 'max').reset_index()
        max_cols = answered.groupby('targName')['pred'].max().to_frame(name = 'max').reset_index()
        unanswered_max_rows = unanswered.merge(max_rows, how = 'left', on = 'candName').fillna(0.0)
        unanswered_max_cols = unanswered.merge(max_cols, how = 'left', on = 'targName').fillna(0.0)
        other_vals_temp = [matcher,] + 8*[None,] + [alg] + 6*[None,] + 7*[1.0,]
#         print(list(zip(other_cols, other_vals_temp)))
#         THRESHOLD
        df_rb_th_P_dynamic = unanswered[unanswered[alg] >= last_P_pred][['correspondence', 'real']]
        df_rb_th_P_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_th_P_dynamic.index)
        df_rb_th['P_dynamic'] = pd.concat([df_rb_th['P_dynamic'], df_rb_th_P_dynamic])
        df_rb_th_P_real = unanswered[unanswered[alg] >= last_P_real][['correspondence', 'real']]
        df_rb_th_P_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_th_P_real.index)
        df_rb_th['P_real'] = pd.concat([df_rb_th['P_real'], df_rb_th_P_real])
        df_rb_th_F_dynamic = unanswered[unanswered[alg] >= 0.5*last_F_pred][['correspondence', 'real']]
        df_rb_th_F_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_th_F_dynamic.index)
        df_rb_th['F_dynamic'] = pd.concat([df_rb_th['F_dynamic'], df_rb_th_F_dynamic])
        df_rb_th_F_real = unanswered[unanswered[alg] >= 0.5*last_F_real][['correspondence', 'real']]
        df_rb_th_F_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_th_F_real.index)
        df_rb_th['F_real'] = pd.concat([df_rb_th['F_real'], df_rb_th_F_real])
#         MAXDELTA        
        df_rb_maxd_P_dynamic = unanswered_max_rows[unanswered_max_rows[alg] >= 
                                                   unanswered_max_rows['max'] - last_P_pred][['correspondence', 'real']]
        df_rb_maxd_P_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxd_P_dynamic.index)
        df_rb_maxd['P_dynamic'] = pd.concat([df_rb_maxd['P_dynamic'], df_rb_maxd_P_dynamic])
        df_rb_maxd_P_real = unanswered_max_rows[unanswered_max_rows[alg] >= 
                                                   unanswered_max_rows['max'] - last_P_real][['correspondence', 'real']]
        df_rb_maxd_P_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxd_P_real.index)
        df_rb_maxd['P_real'] = pd.concat([df_rb_maxd['P_real'], df_rb_maxd_P_real])
        df_rb_maxd_F_dynamic = unanswered_max_rows[unanswered_max_rows[alg] >= 
                                                   unanswered_max_rows['max'] - 0.5*last_F_pred][['correspondence', 'real']]
        df_rb_maxd_F_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxd_F_dynamic.index)
        df_rb_maxd['F_dynamic'] = pd.concat([df_rb_maxd['F_dynamic'], df_rb_maxd_F_dynamic])
        df_rb_maxd_F_real = unanswered_max_rows[unanswered_max_rows[alg] >= 
                                                   unanswered_max_rows['max'] - 0.5*last_F_real][['correspondence', 'real']]
        df_rb_maxd_F_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxd_F_real.index)
        df_rb_maxd['F_real'] = pd.concat([df_rb_maxd['F_real'], df_rb_maxd_F_real])   
#         MAXDELTA COLUMN        
        df_rb_maxdcol_P_dynamic = unanswered_max_cols[unanswered_max_cols[alg] >= 
                                                   unanswered_max_cols['max'] - last_P_pred][['correspondence', 'real']]
        df_rb_maxdcol_P_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxdcol_P_dynamic.index)
        df_rb_maxdcol['P_dynamic'] = pd.concat([df_rb_maxdcol['P_dynamic'], df_rb_maxdcol_P_dynamic])
        df_rb_maxdcol_P_real = unanswered_max_cols[unanswered_max_cols[alg] >= 
                                                   unanswered_max_cols['max'] - last_P_real][['correspondence', 'real']]
        df_rb_maxdcol_P_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxdcol_P_real.index)
        df_rb_maxdcol['P_real'] = pd.concat([df_rb_maxdcol['P_real'], df_rb_maxdcol_P_real])
        df_rb_maxdcol_F_dynamic = unanswered_max_cols[unanswered_max_cols[alg] >= 
                                                   unanswered_max_cols['max'] - 0.5*last_F_pred][['correspondence', 'real']]
        df_rb_maxdcol_F_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxdcol_F_dynamic.index)
        df_rb_maxdcol['F_dynamic'] = pd.concat([df_rb_maxdcol['F_dynamic'], df_rb_maxdcol_F_dynamic])
        df_rb_maxdcol_F_real = unanswered_max_cols[unanswered_max_cols[alg] >= 
                                                   unanswered_max_cols['max'] - 0.5*last_F_real][['correspondence', 'real']]
        df_rb_maxdcol_F_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxdcol_F_real.index)
        df_rb_maxdcol['F_real'] = pd.concat([df_rb_maxdcol['F_real'], df_rb_maxdcol_F_real]) 
#         DOMINANTS       
        df_rb_dom_P_dynamic = unanswered_max_rows[(unanswered_max_rows[alg] >= 
                                                unanswered_max_rows['max'] - last_P_pred) &
                                                (unanswered_max_cols[alg] >= 
                                                unanswered_max_cols['max'] - last_P_pred)][['correspondence', 'real']]
        df_rb_dom_P_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_dom_P_dynamic.index)
        df_rb_dom['P_dynamic'] = pd.concat([df_rb_dom['P_dynamic'], df_rb_dom_P_dynamic])
        df_rb_dom_P_real = unanswered_max_rows[(unanswered_max_rows[alg] >= 
                                                unanswered_max_rows['max'] - last_P_real) &
                                                (unanswered_max_cols[alg] >= 
                                                unanswered_max_cols['max'] - last_P_real)][['correspondence', 'real']]
        df_rb_dom_P_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_dom_P_real.index)
        df_rb_dom['P_real'] = pd.concat([df_rb_dom['P_real'], df_rb_dom_P_real])
        df_rb_dom_F_dynamic = unanswered_max_rows[(unanswered_max_rows[alg] >= 
                                                unanswered_max_rows['max'] - 0.5*last_F_pred) &
                                                (unanswered_max_cols[alg] >= 
                                                unanswered_max_cols['max'] - 0.5*last_F_pred)][['correspondence', 'real']]
        df_rb_dom_F_dynamic[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_dom_F_dynamic.index)
        df_rb_dom['F_dynamic'] = pd.concat([df_rb_dom['F_dynamic'], df_rb_dom_F_dynamic])
        df_rb_dom_F_real = unanswered_max_rows[(unanswered_max_rows[alg] >= 
                                                unanswered_max_rows['max'] - 0.5*last_F_real) &
                                                (unanswered_max_cols[alg] >= 
                                                unanswered_max_cols['max'] - 0.5*last_F_real)][['correspondence', 'real']]
        df_rb_dom_F_real[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_dom_F_real.index)
        df_rb_dom['F_real'] = pd.concat([df_rb_dom['F_real'], df_rb_dom_F_real]) 
#         STATICS:
        for num in thresholds:
#         THRESHOLD:
            df_rb_th_temp = unanswered[unanswered[alg] >= num][['correspondence', 'real']]
            df_rb_th_temp[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_th_temp.index)
            df_rb_th[str(num)] = pd.concat([df_rb_th[str(num)], df_rb_th_temp])
#             MAXDELTA
            df_rb_maxd_temp = unanswered_max_rows[unanswered_max_rows[alg] >= 
                                                       unanswered_max_rows['max'] - num][['correspondence', 'real']]
            df_rb_maxd_temp[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxd_temp.index)
            df_rb_maxd[str(num)] = pd.concat([df_rb_maxd[str(num)], df_rb_maxd_temp])
#             MAXDELTA COLUMN
            df_rb_maxdcol_temp = unanswered_max_cols[unanswered_max_cols[alg] >= 
                                                       unanswered_max_cols['max'] - num][['correspondence', 'real']]
            df_rb_maxdcol_temp[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_maxdcol_temp.index)
            df_rb_maxdcol[str(num)] = pd.concat([df_rb_maxdcol[str(num)], df_rb_maxdcol_temp])
#             DOMINANTS
            df_rb_dom_temp = unanswered_max_rows[(unanswered_max_rows[alg] >= 
                                                unanswered_max_rows['max'] - num) &
                                                (unanswered_max_cols[alg] >= 
                                                unanswered_max_cols['max'] - num)][['correspondence', 'real']]
            df_rb_dom_temp[other_cols] = pd.DataFrame([other_vals_temp], index = df_rb_dom_temp.index)
            df_rb_dom[str(num)] = pd.concat([df_rb_dom[str(num)], df_rb_dom_temp])  
    break

In [ ]:
def fix_df_rb(df_rb_0, df):
    clfs = df['clf'].unique().tolist()
    regs = df['reg'].unique().tolist()
    new_df_rb = None
    for clf in clfs:
        for reg in regs:
            temp_df_rb = df_rb_0.copy() 
            temp_df_rb['alg'] = df_rb_0['alg_matcher'] + ' ' + clf + ' ' + reg
            new_df_rb = pd.concat([new_df_rb, temp_df_rb])
    return new_df_rb

In [ ]:
for RB, df_rb in df_rb_th.items():
    df_rb = fix_df_rb(df_rb, df_hp)
    full_df = pd.concat([df_hp, df_rb])
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_R': 'pred'}), 'hp_target_R_rb_threshold_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_static': 'pred'}), 'hp_target_P_static_rb_threshold_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_dynamic': 'pred'}), 'hp_target_P_dynamic_rb_threshold_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_static': 'pred'}), 'hp_target_F_static_rb_threshold_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_dynamic': 'pred'}), 'hp_target_F_dynamic_rb_threshold_' + RB)
    
for RB, df_rb in df_rb_maxd.items():
    full_df = pd.concat([df_hp, df_rb])
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_R': 'pred'}), 'hp_target_R_rb_maxd_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_static': 'pred'}), 'hp_target_P_static_rb_maxd_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_dynamic': 'pred'}), 'hp_target_P_dynamic_rb_maxd_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_static': 'pred'}), 'hp_target_F_static_rb_maxd_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_dynamic': 'pred'}), 'hp_target_F_dynamic_rb_maxd_' + RB)

for RB, df_rb in df_rb_maxdcol.items():
    full_df = pd.concat([df_hp, df_rb])
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_R': 'pred'}), 'hp_target_R_rb_maxdcol_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_static': 'pred'}), 'hp_target_P_static_rb_maxdcol_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_dynamic': 'pred'}), 'hp_target_P_dynamic_rb_maxdcol_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_static': 'pred'}), 'hp_target_F_static_rb_maxdcol_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_dynamic': 'pred'}), 'hp_target_F_dynamic_rb_maxdcol_' + RB)

for RB, df_rb in df_rb_dom.items():
    full_df = pd.concat([df_hp, df_rb])
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_R': 'pred'}), 'hp_target_R_rb_dom_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_static': 'pred'}), 'hp_target_P_static_rb_dom_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_P_dynamic': 'pred'}), 'hp_target_P_dynamic_rb_dom_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_static': 'pred'}), 'hp_target_F_static_rb_dom_' + RB)
    _ = evaluate(full_df.rename(columns={'pred': 'temp', 'pred_F_dynamic': 'pred'}), 'hp_target_F_dynamic_rb_dom_' + RB)